# Goal

Find CyCOGs corresponding to genes that have been observed to be up-regulated by MED4 _Prochlorococcus_ in response to viral infection in a laboratory setting. Reference: [Lindell et al. (2007)](https://www.nature.com/articles/nature06130)

## Inputs:
- Supplemental Table of MED4 genes up-regulated in response to viral infection (from Lindell et al. (2007))
- CyCOG to MED4 gene mapping file

## Output
- Dataframe matching each up-regulated gene with its corresponding CyCOG


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import supplemental table of MED4 up-regulated genes
#document how this table was made


#MED4 up-regulated genes 
lindell =pd.read_csv('.\lindell.csv')


#importing Cycoc to MED4 gene mapping 
med4_mapping = pd.read_csv('.\MED4_cycog6_table.tsv',
                      names = ['Gene Name', 'MED4', 'CyCOG'],
                      sep = '\t')

#pull in annotated_cycog data as "C:\Users\jdhru\Downloads\pro-clusters.xlsx"well
cycog_df = pd.read_csv('./cycog_annotations.csv')

#blasks clusters
cluster_13 = pd.read_excel('./pro-clusters.xlsx', sheet_name = 'cluster13')
cluster_4 = pd.read_excel('./pro-clusters.xlsx', sheet_name = 'cluster4')
cluster_2 = pd.read_excel('./pro-clusters.xlsx', sheet_name = 'cluster2')
cluster_15 = pd.read_excel('./pro-clusters.xlsx', sheet_name = 'cluster15')


#Doron: https://www.science.org/doi/suppl/10.1126/science.aar4120/suppl_file/aar4120_tabless1-s5.xlsx 

#known defence families 
Doron1 = pd.read_excel('aar4120_tabless1-s5.xlsx', sheet_name = 'S1 - Known defense families' )
#putative denfence systems 
Doron3 = pd.read_excel('aar4120_tabless1-s5.xlsx', sheet_name = 'S3 - candidate systems')

In [3]:
#preproess cluters 

#join all clusters together, keep only those genes with % bootstrap of .5 or higher 
cluster_13.insert(0, "cluster", 13)
cluster_4.insert(0, "cluster", 4)
cluster_2.insert(0, "cluster", 2)
cluster_15.insert(0, "cluster", 15)
all_clusters = pd.concat([cluster_13,cluster_4,cluster_2,cluster_15])
all_clusters = all_clusters[all_clusters['% bootstraps'] > 0.5]

In [4]:
#match mapping file CyCOG formatting to annotated cycog file from blasks
med4_mapping['CyCOG'] = med4_mapping['CyCOG'].str.replace("CyCOG_", '').astype('int64')
print(len(med4_mapping))
med4_mapping.head()

1956


,Gene Name,MED4,CyCOG
0,PMM0001,MED4_2606841238,60000408
1,PMM0002,MED4_2606841239,60000496
2,PMM0003,MED4_2606841240,60000076
3,PMM0004,MED4_2606841241,60000579
4,PMM0005,MED4_2606841242,60000069


In [5]:
#add in cycog annotation data to mapping file
cycog_df = cycog_df.rename(columns={"CyCOGID":"CyCOG"})
cycog_df['CyCOG'] = cycog_df['CyCOG'].astype('int64')
med4_mapping['CyCOG'] = med4_mapping['CyCOG'].astype('int64')
med4_mapping = pd.merge(med4_mapping, cycog_df, how="left", on="CyCOG")
med4_mapping.head()

,Gene Name,MED4,CyCOG,TotalRefs,Prochlorococcus,Synechococcus,Virocell,Virus,5.1A-CRD2,5.1A-II,...,NRefsKO,NRefsOtherKO,COGID,DescriptionCOG,NRefsCOG,NRefsOtherCOG,PfamID,DescriptionPfam,NRefsPfam,NRefsOtherPfam
0,PMM0001,MED4_2606841238,60000408,467,412,53,2,0,3,6,...,453.0,0.0,COG0592,DNA polymerase III sliding clamp (beta) subuni...,446.0,0.0,pfam00712,DNA_pol3_beta,388.0,78.0
1,PMM0002,MED4_2606841239,60000496,460,407,51,2,0,3,6,...,NaN,NaN,COG1873,"Sporulation protein YlmC, PRC-barrel domain fa...",1.0,0.0,pfam05239,PRC,263.0,0.0
2,PMM0003,MED4_2606841240,60000076,506,445,59,2,0,4,8,...,464.0,0.0,COG0046,Phosphoribosylformylglycinamidine (FGAM) synth...,460.0,0.0,pfam02769,AIRS_C,473.0,25.0
3,PMM0004,MED4_2606841241,60000579,467,439,26,2,0,4,8,...,450.0,0.0,COG0034,Glutamine phosphoribosylpyrophosphate amidotra...,442.0,0.0,pfam13522,GATase_6,441.0,12.0
4,PMM0005,MED4_2606841242,60000069,530,470,58,2,0,4,7,...,456.0,0.0,COG0188,"DNA gyrase/topoisomerase IV, subunit A",425.0,0.0,pfam00521,DNA_topoisoIV,486.0,17.0


In [6]:
#map Lindell data to respective Cycogs 
lindell = pd.merge(lindell, med4_mapping, how='left', on='Gene Name' )
print(len(lindell))
lindell.head()

45


,Gene Name,Possible product,Function,0 hr,1 hr,2 hr,3 hr,4 hr,5 hr,6 hr,...,NRefsKO,NRefsOtherKO,COGID,DescriptionCOG,NRefsCOG,NRefsOtherCOG,PfamID,DescriptionPfam,NRefsPfam,NRefsOtherPfam
0,PMM0549,csoS1 carboxysome shell protein 1,carbon fixation,1.70,1.31,-1.22,-1.59,1.76,-2.07,-2.03,...,NaN,NaN,NaN,NaN,NaN,NaN,pfam00936,BMC,612.0,0.0
1,PMM0550,rbcL rubisco large subunit,carbon fixation,1.79,2.01,1.38,1.18,-1.41,-1.69,-1.62,...,425.0,0.0,COG1850,"Ribulose 1,5-bisphosphate carboxylase, large s...",419.0,0.0,pfam00016,RuBisCO_large,432.0,8.0
2,PMM0551,rbcS rubisco small subunit,carbon fixation,1.63,1.85,1.36,1.17,-1.43,-1.67,-1.62,...,422.0,0.0,COG4451,Ribulose bisphosphate carboxylase small subunit,419.0,0.0,pfam00101,RuBisCO_small,424.0,0.0
3,PMM0815,hli19/09 high-light inducible stress response ...,NaN,1.15,1.23,-1.27,-1.45,-1.25,-1.38,-1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,pfam00504,Chloroa_b-bind,716.0,0.0
4,PMM1396,hli19/09 high-light inducible stress response ...,NaN,1.15,1.23,-1.27,-1.45,-1.25,-1.38,-1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#see which cycogs are in lindells paper
cycogs_in_lindell = all_clusters[all_clusters['cycog'].isin(lindell['CyCOG'])]
cycogs_in_lindell

,cluster,gene,cycog,annotation,weight,% bootstraps
2,15,C,60000134,RNA polymerase nonessential primary-like sigma...,0.286855,0.923333


In [8]:
#distribution of cycog clusters 
cycog_dist = pd.DataFrame(all_clusters['cycog'].value_counts())
cycog_dist.reset_index(inplace=True)
cycog_dist.rename(columns={'index':'cycog','cycog':'counts'}, inplace=True)
cycog_dist[cycog_dist['counts']>1]


,cycog,counts
0,60002463,3
1,60001830,3
2,60002480,2
3,60000381,2
4,60000312,2
5,60001461,2
6,60000703,2
7,60007240,2
8,60001466,2
9,60001259,2
